In [1]:
import tflearn

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
import nltk

In [3]:
import tensorflow as tf
import numpy as np
import random
import json

In [4]:
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
with open('Desktop/Poke.json') as json_data:
    intents=json.load(json_data)

In [6]:
intents

{'intents': [{'context_set': '',
   'images': [' '],
   'patterns': ['Hi',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day',
    'Hey'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'tag': 'greeting'},
  {'images': [' '],
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.'],
   'tag': 'goodbye'},
  {'images': [' '],
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'tag': 'thanks'},
  {'images': [' '],
   'patterns': ['How can u help?',
    'What can you do for me?',
    'You are here for?',
    'What is your name?',
    'Your name?'],
   'responses': ['I am Pokebot and I am here to give you info about 14 different types of pokemons like pikachu, caterpie, pidgeotto, bulbasaur, charmender, squirtle, krabby, raticat

In [7]:
words=[]
classes=[]
doc=[]
ignore=['?']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w=nltk.word_tokenize(pattern)
        words.extend(w)
        doc.append((w,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [8]:
words1=[stemmer.stem(w.lower()) for w in words if w not in ignore]
words1=sorted(list(set(words1)))
classes=sorted(list(set(classes)))

In [9]:
words1

["'s",
 'about',
 'anyon',
 'ar',
 'balbasa',
 'bulbasa',
 'bye',
 'can',
 'caterpy',
 'charmand',
 'day',
 'do',
 'for',
 'good',
 'goodby',
 'haunt',
 'hello',
 'help',
 'her',
 'hey',
 'hi',
 'how',
 'is',
 'krabby',
 'lapra',
 'lat',
 'me',
 'muk',
 'nam',
 'pidgeotto',
 'pik',
 'pikachu',
 'primeap',
 'rat',
 'see',
 'snorlax',
 'squirtl',
 'tauro',
 'thank',
 'that',
 'ther',
 'u',
 'what',
 'who',
 'yo',
 'you']

In [10]:
training=[]
output=[]
output_empty=[0]*len(classes)


In [11]:
for d in doc:
    bag=[]
    pattern_words=d[0]
    pattern_words=[stemmer.stem(word.lower()) for word in pattern_words]
    for w in words1:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row=list(output_empty)
    output_row[classes.index(d[1])]=1
    training.append([bag,output_row])

In [12]:
random.shuffle(training)

In [13]:
training=np.array(training)

In [14]:
train_X=list(training[:,0])
train_Y=list(training[:,1])

In [15]:
tf.reset_default_graph()

In [16]:
net=tflearn.input_data(shape=[None,len(train_X[0])])

In [17]:
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,len(train_Y[0]),activation='softmax')

In [18]:
net=tflearn.regression(net)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [19]:
model=tflearn.DNN(net,tensorboard_dir='tflearn_logs')

In [21]:
#model.fit(train_X,train_Y,n_epoch=2500,batch_size=16,show_metric=True)

In [21]:
#model.save('model4_9999.tflearn')

INFO:tensorflow:C:\Users\KIIT\model4_9999.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [20]:
import pickle

In [23]:
#pickle.dump({'words':words1,'classes':classes,'train_x':train_X,'train_y':train_Y},open("training_data4","wb"))

In [21]:
model.load('model4_9999.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\KIIT\model4_9999.tflearn


In [22]:
data=pickle.load(open("training_data4","rb"))

In [23]:
words=data['words']
classes=data['classes']
train_x=data['train_x']
train_y=data['train_y']

In [24]:
with open('Desktop/Poke.json') as json_data:
    intents=json.load(json_data)

In [25]:
def clean_up(sentence):
    sentence_words=nltk.word_tokenize(sentence)
    sentence_words=[stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence,words):
    sentence_words=clean_up(sentence)
    bag=[0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w==s:
                bag[i]=1
                
    return np.array(bag)

def classify(sentence):
    results=model.predict([bow(sentence,words)])[0]
    results=[[i,r] for i,r in enumerate(results) if r>0.40]
    results.sort(key=lambda x:x[1],reverse=True)
    return_list=[]
    for r in results:
        return_list.append((classes[r[0]],r[1]))
    return return_list

def response(sentence):
    results=classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                if i['tag']==results[0][0]:
                    return (random.choice(i['responses']),i['images'][0])
            results.pop(0)
    return ('res','url')
        

In [29]:
#from PIL import Image

In [26]:
import tkinter as tk
from tkinter import *
from PIL import ImageTk
from urllib.request import urlopen
#from io import BytesIO
import PIL.Image

In [27]:
'''def quit():
    base.destroy()
    exit()'''
#import PIL.Image
msg=''


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        
        ChatLog.config(state=NORMAL)
        ChatLog.delete("0.0",END)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
        (res,url)=response(msg)
        
        if( res!=None and url!=' ') and (res!='res' and url!='url'):
            ''' = "https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTR-E-nsp-BcjJ1-Ip5V375qH5dLQ2sTFFi4w&usqp=CAU"
            u = urlopen(URL)
            raw_data = u.read()
            u.close()
            im = Image.open(BytesIO(raw_data))
            
            label.pack_forget()
            i=PIL.Image.open(urlopen(url))
            photo = ImageTk.PhotoImage(i)
            #abel = tk.Label(image=photo)
            label.image = photo
            
            
            #abel.place(x=10,y=60)
            #abel.destroy()'''
            i=PIL.Image.open(urlopen(url))
            photo = ImageTk.PhotoImage(i)
            label.configure(image=photo)
            label.image=photo
            label.place(x=500,y=170)

            ChatLog.insert(END, "PokeBot: " + res + '\n\n')
            ChatLog.config(state=DISABLED)
            ChatLog.yview(END)
        elif res!=None and url==' ':
            i=PIL.Image.open(urlopen("https://www.colorhexa.com/add8e6.png"))
            photo = ImageTk.PhotoImage(i)
            label.configure(image=photo)
            label.image=photo
            label.place(x=500,y=170)
            label.pack_forget()
            ChatLog.insert(END, "PokeBot: " + res + '\n\n')
            ChatLog.config(state=DISABLED)
            ChatLog.yview(END)
        else:
            i=PIL.Image.open(urlopen("https://www.colorhexa.com/add8e6.png"))
            photo = ImageTk.PhotoImage(i)
            label.configure(image=photo)
            label.image=photo
            label.place(x=500,y=170)
            label.pack_forget()
            ChatLog.insert(END, "PokeBot: " + "Please be more specific I couldn't get you." + '\n\n')
            ChatLog.config(state=DISABLED)
            ChatLog.yview(END)

base = Tk()
base.title("Pokemon Trainees")
base.geometry("1200x600")
base.resizable(width=False, height=False)
#Create Chat window
ChatLog = Text(base, bd=0, bg="#ADD8E6", height="8", width="50", font="Arial",)
ChatLog.config(state=DISABLED)
#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview)
ChatLog['yscrollcommand'] = scrollbar.set
#Create Button to send message
'''i=PIL.Image.open(urlopen("https://www.colorhexa.com/add8e6.png"))
photo = ImageTk.PhotoImage(i)'''
label = tk.Label(base,image=None)
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="red", activebackground="white",fg='#ffffff',
                    command= send )
#Create the box to enter message
EntryBox = Text(base, bd=0, bg="#ADD8E6",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)
#Place all components on the screen
#scrollbar.place(x=1180,height=386,width=20)
ChatLog.place(x=6,y=6, height=510,width=1175)
EntryBox.place(x=128, y=520,height=60, width=265)
SendButton.place(x=6, y=520,height=60)
#Button(base,text="Exit",command=quit).pack()
base.mainloop()